In [ ]:
!pip install transformers accelerate
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset, get_dataset_config_names, Dataset
from wordcloud import WordCloud
import evaluate

from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding

from transformers import create_optimizer
import tensorflow as tf

from transformers import TFAutoModelForSequenceClassification

from transformers import create_optimizer
import tensorflow as tf

from transformers.keras_callbacks import KerasMetricCallback, PushToHubCallback

In [ ]:
df_train = pd.read_csv("train2.csv")
df_train.rename(columns={'score': 'label'}, inplace=True)
df_test = pd.read_csv("test.csv")
df_test.rename(columns={'score': 'label'}, inplace=True)
df_test.sample(10)

,Unnamed: 0,review_id,product_id,reviewer_id,notes,body,title,lang,product,opinion,bodylen,titlelen,comment,label
2991,2991,fr_0800834,product_fr_0541396,reviewer_fr_0684123,3,Cet onduleur est bruyant et Amazon refuse de p...,Bruyant contrairement à ce que j'ai lu...,fr,pc,mixed,85,41,bruyant contrairement à ce que jai lu cet ondu...,1
1349,1349,fr_0653055,product_fr_0613334,reviewer_fr_0774308,2,J’ai commandé des ondulés comme sur la photo e...,Pinces.,fr,home,non satisfied,133,7,pinces j’ai commandé des ondulés comme sur la ...,0
1550,1550,fr_0913862,product_fr_0627579,reviewer_fr_0187736,5,Livraison rapide et du bon matériel,Bon produit,fr,electronics,satisfied,35,11,bon produit livraison rapide et du bon matériel,2
2675,2675,fr_0249904,product_fr_0910475,reviewer_fr_0972847,2,Les lacets on cassé des les première utilisati...,Nul,fr,shoes,non satisfied,50,3,nul les lacets on cassé des les première utili...,0
1818,1818,fr_0386181,product_fr_0627084,reviewer_fr_0370533,1,C est une pourriture fini le vendeur d ailleur...,Arnaque,fr,beauty,non satisfied,525,7,arnaque c est une pourriture fini le vendeur d...,0
1107,1107,fr_0899447,product_fr_0568614,reviewer_fr_0017804,3,"Ces chaussettes sont bien chaudes, mais malheu...",Piquent,fr,apparel,mixed,186,7,piquent ces chaussettes sont bien chaudes mais...,1
2510,2510,en_0354187,product_en_0301200,reviewer_en_0348382,5,It is easy to use due to its faster heating. e...,Easy to use,en,kitchen,satisfied,157,11,easy to use it is easy to use due to its faste...,2
353,353,fr_0241234,product_fr_0899397,reviewer_fr_0788276,5,Pour redonner la qualité à mes couteau qui n'é...,C'est un produit magnifique facile à utilisé,fr,home,satisfied,81,44,cest un produit magnifique facile à utilisé po...,2
2616,2616,fr_0874424,product_fr_0209904,reviewer_fr_0757909,3,"Produit conforme aux photo, matière agréable e...",Bien mais sans plus,fr,other,mixed,363,19,bien mais sans plus produit conforme aux photo...,1
2551,2551,en_0201396,product_en_0191409,reviewer_en_0019652,2,"Fits nice, marble is faded and doesn’t look go...",Faded marble print,en,pc,mixed,286,18,faded marble print fits nice marble is faded a...,1


In [ ]:
train_set = df_train.loc[:, ["comment", "label"]]
train_set = Dataset.from_pandas(train_set)

test_set = df_test.loc[:, ["comment", "label"]]
test_set = Dataset.from_pandas(test_set)
test_set

Dataset({
    features: ['comment', 'label'],
    num_rows: 3000
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")

In [ ]:
def preprocess_function(element):
    return tokenizer(element["comment"], truncation=True)

In [ ]:
tokenized_train_set = train_set.map(preprocess_function, batched=True)
tokenized_test_set = test_set.map(preprocess_function, batched=True)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
id2label = {0: "non satisfied", 1: "mixed", 2: "satisfied"}
label2id = {"non satisfied": 0, "mixed": 1, "satisfied": 2}

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-multilingual-cased", num_labels=3, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
batchsize = 16

tf_train_set = model.prepare_tf_dataset(
    tokenized_train_set,
    shuffle=True,
    batch_size=batchsize,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_test_set,
    shuffle=False,
    batch_size=batchsize,
    collate_fn=data_collator,
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
num_epochs = 3
lr = 5e-5
batches_per_epoch = len(tokenized_train_set) // batchsize
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=lr, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
model.compile(optimizer=optimizer)

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-c

In [ ]:
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_test_set)

push_to_hub_callback = PushToHubCallback(
    output_dir="my_awesome_model2",
    tokenizer=tokenizer,
)
callbacks = [metric_callback, push_to_hub_callback]

/content/my_awesome_model2 is already a clone of https://huggingface.co/Josias-Ounsinli/my_awesome_model2. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=num_epochs, callbacks=callbacks)

Epoch 1/3
1875/1875 [==============================] - 610s 306ms/step - loss: 0.7160 - val_loss: 0.6230 - accuracy: 0.7303
Epoch 2/3
1875/1875 [==============================] - 554s 295ms/step - loss: 0.5531 - val_loss: 0.5978 - accuracy: 0.7363
Epoch 3/3
1875/1875 [==============================] - 548s 292ms/step - loss: 0.3952 - val_loss: 0.6597 - accuracy: 0.7360


# Making a small test using the model

In [ ]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis', model='Josias-Ounsinli/my_awesome_model2')

Some layers from the model checkpoint at Josias-Ounsinli/my_awesome_model2 were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at Josias-Ounsinli/my_awesome_model2 and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
text = "This is good. However, it has to be improved. At the same time, it's bad."
classifier(text)

[{'label': 'mixed', 'score': 0.6949605941772461}]